# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_data = pd.read_csv("output_data/cities.csv")
cities_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Yellowknife,62.4560,-114.3525,5.87,60,100,2.68,CA,1633720500
1,Atuona,-9.8000,-139.0333,25.53,72,6,7.06,PF,1633720640
2,Mataura,-46.1927,168.8643,9.71,100,100,1.35,NZ,1633720640
3,Narsaq,60.9167,-46.0500,6.12,50,2,1.71,GL,1633720641
4,Bathsheba,13.2167,-59.5167,28.34,78,40,5.14,BB,1633720641


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [20]:
# Set up locations and weight for the map
locations = cities_data[['Lat', 'Lng']]
weights = cities_data['Humidity']

# Display heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
ideal_cities = cities_data.loc[(cities_data['Max Temp'] <= 24.4) & (cities_data['Humidity'] <= 68) & (cities_data['Wind Speed'] <= 2) & (cities_data['Cloudiness'] <= 0)]
ideal_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,Rzhaksa,52.1338,42.0264,4.71,57,0,1.81,RU,1633720644
105,Mordovo,52.0834,40.7700,5.16,55,0,1.97,RU,1633720687
136,Heraclea,37.5133,34.0467,14.19,31,0,0.84,TR,1633720701
155,Bodri,23.1667,81.4333,22.97,68,0,1.81,IN,1633720711
357,Marquard,-28.6645,27.4305,16.03,22,0,1.58,ZA,1633720800
425,Niğde,37.8333,34.7500,10.05,29,0,0.62,TR,1633720833
476,Korla,41.7597,86.1469,6.14,33,0,1.72,CN,1633720858
513,Milas,37.3164,27.7839,15.64,67,0,1.54,TR,1633720876
527,Tingi,-11.3000,35.0333,19.43,51,0,0.78,TZ,1633720883
558,Jurm,36.8648,70.8342,10.31,47,0,0.85,AF,1633720897


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [170]:
# Create the hotels dataframe with a column for Hotel Name
hotel_df = pd.DataFrame(columns=['Hotel Name'])

# Set up the base url to query a nearby search in google API
nearby_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
place_url = "https://maps.googleapis.com/maps/api/geocode/json?latlng="

In [178]:
# Loop through the cities dataframe, using the latitude and longitude data for the location parameter in the API request.
# Store the hotel name, latitude and longitude in lists. Use the latitude and longitude from the query to 
# perform another query for the hotel's city and country. Store each in separate lists
hotel_names = []
cities = []
countries = []
responses = []
for index, row in ideal_cities.iterrows():
    latitude = row[1]
    longitude = row[2]
    hotel_query_url = f"{nearby_url}?keyword=hotel&location={latitude}%2C{longitude}&radius=5000&type=hotel&key={g_key}"
    hotel_response = requests.get(hotel_query_url).json()
    try:
        hotel_names.append(hotel_response['results'][0]['name'])
        hotel_lat = hotel_response['results'][0]['geometry']['location']['lat']
        hotel_lng = hotel_response['results'][0]['geometry']['location']['lng']
        place_query_url = f"{place_url}{hotel_lat},{hotel_lng}&result_type=locality%7Ccountry&key={g_key}"
        place_response = requests.get(place_query_url).json()
        try:
            cities.append(place_response['results'][0]['address_components'][0]['long_name'])
        except:
            cities.append('No city found')
        try:
            countries.append(place_response['results'][0]['address_components'][3]['long_name'])
        except:
            countries.append('No country found')
    except:
#         print('no place found')
        hotel_names.append('No hotel within 5000 meters!')
#     responses.append(hotel_response)

# Use the lists to fill in the hotels dataframe
# hotel_df['Hotel Name'] = hotel_names
# hotel_df['City'] = cities
# hotel_df['Country'] = countries
# hotel_df
# Loop through the hotels dataframe find the cities and countries the hotel is in

# Plot the 

In [180]:
cities

['Rzhaksa',
 'Belceağaç',
 'Marquard',
 'Bayingolin Mongol Autonomous Prefecture',
 'Milas']

In [150]:
hotel_lat = responses[0]['results'][0]['geometry']['location']['lat']
hotel_lng = responses[0]['results'][0]['geometry']['location']['lng']
hotel_lng

42.0174166

In [152]:
place_url = "https://maps.googleapis.com/maps/api/geocode/json?latlng="
place_query_url = f"{place_url}{hotel_lat},{hotel_lng}&result_type=locality%7Ccountry&key={g_key}"
place_response = requests.get(place_query_url).json()

{'plus_code': {'compound_code': '42Q8+6X Rzhaksa, Tambov Oblast, Russia',
  'global_code': '9H4442Q8+6X'},
 'results': [{'address_components': [{'long_name': 'Rzhaksa',
     'short_name': 'Rzhaksa',
     'types': ['locality', 'political']},
    {'long_name': 'Rzhaksinsky District',
     'short_name': 'Rzhaksinsky District',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'Tambov Oblast',
     'short_name': 'Tambov Oblast',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'Russia',
     'short_name': 'RU',
     'types': ['country', 'political']}],
   'formatted_address': 'Rzhaksa, Tambov Oblast, Russia',
   'geometry': {'bounds': {'northeast': {'lat': 52.182011, 'lng': 42.1361589},
     'southwest': {'lat': 52.1198941, 'lng': 41.9892339}},
    'location': {'lat': 52.1342381, 'lng': 42.02441109999999},
    'location_type': 'APPROXIMATE',
    'viewport': {'northeast': {'lat': 52.182011, 'lng': 42.1361589},
     'southwest': {'la

In [163]:
city = place_response['results'][0]['address_components'][0]['long_name']
country = place_response['results'][0]['address_components'][3]['long_name']

'Russia'

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
